In [ ]:
# Import necessary libraries
import pandas as pd
from textblob import TextBlob
import kagglehub
from kagglehub import KaggleDatasetAdapter

# Set the path to the file you'd like to load
file_path = "amazon.csv"

# Load the Kaggle dataframe
amazon_df_raw = kagglehub.dataset_load(
  KaggleDatasetAdapter.PANDAS,
  "karkavelrajaj/amazon-sales-dataset",
  file_path,)

amazon_df = amazon_df_raw.copy()

amazon_df.head()

In [ ]:
## Alternative method (disabled by default)
## To run this cell, delete the first "#" on each line

## Import necessary libraries
#import os
#import shutil
#import pandas as pd
#from textblob import TextBlob
#import kagglehub

## Download Kaggle dataset to cache
#download_path = kagglehub.dataset_download("karkavelrajaj/amazon-sales-dataset", path="amazon.csv")

## Define target path and make sure the target directory exists
#target_path = "data/amazon.csv"
#os.makedirs(os.path.dirname(target_path), exist_ok = True)

## Move the downloaded file to the target path
#shutil.move(download_path, target_path)

## Read CSV file and load the dataframe
#amazon_df_raw = pd.read_csv(target_path)
#amazon_df = amazon_df_raw.copy()

#amazon_df.head()

In [ ]:
# Drop "img_link", "product_link", and other columns
amazon_df = amazon_df.drop(["user_id", "user_name", "review_id", "img_link", "product_link"], axis = 1)
amazon_df.head()

In [ ]:
# Perform a quick clean if there are fewer than 2% rows with empty cells
def quick_clean(df):
    threshold = 2 # Default threshold set to 2%. Change the value where necessary. 
    df_qclean = df.dropna()
    original_count = df.shape[0]
    cleaned_count = df_qclean.shape[0]

    percentage_dropped = (original_count - cleaned_count) / original_count * 100

    print("Number of items (original):", original_count)
    print("Number of items (quick clean):", cleaned_count)
    print(f"Percentage dropped: {round(percentage_dropped, 2)}%")

    if percentage_dropped < threshold:
        print(f"Quick clean applied to the DataFrame. {round(percentage_dropped, 2)}% of rows dropped.")
        return df_qclean
    else:
        print("More sophisticated cleaning methods needed.")
        return df

# Apply quick clean function
amazon_df = quick_clean(amazon_df)

In [ ]:
# Check data types
amazon_df.dtypes

In [ ]:
# Find unique data types in each column
amazon_df.apply(lambda col: col.map(type).unique())

In [ ]:
# Check the first 20 unique items in the "category" column
amazon_df["category"].unique()[0: 20]

In [ ]:
# Basic transformation of the "category" column
amazon_df.loc[:, "category"] = amazon_df["category"].str.replace("&", " & ").str.replace(",", ", ")

In [ ]:
# Split categories and add to the original DataFrame
category_split_df = amazon_df["category"].str.split("|", expand=True)
category_split_df.fillna("none", inplace=True)
category_split_df.columns = [f"category_l{str(i + 1).zfill(2)}" for i in range(category_split_df.shape[1])]

# Insert the split columns after "category"
category_col_index = amazon_df.columns.get_loc("category")
for i, col in enumerate(category_split_df.columns):
    amazon_df.insert(category_col_index + i + 1, col, category_split_df[col])

amazon_df.head()

In [ ]:
# Check number of "none" in each level of categories and subcategories
for i in range(7):
    category = f"category_l0{i+1}"
    count_none = (amazon_df[category] == "none").sum()
    print(f"Number of \"none\" in category_l0{i + 1}: {count_none}")

In [ ]:
# Category levels without "none" will be further transformed
# Check unique categories and subcategories
for i in range(2):
    category = f"category_l0{i + 1}"
    print(f"Unique items in category_l0{i + 1}: {amazon_df[category].unique()}")

In [ ]:
# Transform categories and subcategories
amazon_df.loc[:, "category_l01"] = amazon_df["category_l01"].str.replace("MusicalInstruments","Musical Instruments") \
.str.replace("OfficeProducts", "Office Products").str.replace("HomeImprovement", "Home Improvement") \
.str.replace("PersonalCare", "Personal Care")
amazon_df.loc[:, "category_l02"] = amazon_df["category_l02"].str.replace("NetworkingDevices", "Networking Devices") \
.str.replace("HomeTheater", "Home Theater").str.replace("HomeAudio", "Home Audio") \
.str.replace("WearableTechnology", "Wearable Technology").str.replace("ExternalDevices", "External Devices") \
.str.replace("DataStorage", "Data Storage").str.replace("GeneralPurposeBatteries", "General Purpose Batteries") \
.str.replace("BatteryChargers", "Battery Chargers").str.replace("OfficePaperProducts", "Office Paper Products") \
.str.replace("CraftMaterials", "Craft Materials").str.replace("OfficeElectronics", "Office Electronics") \
.str.replace("PowerAccessories", "Power Accessories").str.replace("HomeAppliances", "Home Appliances") \
.str.replace("AirQuality", "Air Quality").str.replace("HomeStorage", "Home Storage") \
.str.replace("CarAccessories", "Car Accessories").str.replace("HomeMedicalSupplies", "Home Medical Supplies")

for i in range(2):
    category = f"category_l0{i + 1}"
    print(f"Unique items in category_l0{i + 1}: {amazon_df[category].unique()}")

In [ ]:
# Categories levels with "none" are dropped
for i in range(3, 8):
    category = f"category_l0{i}"
    amazon_df = amazon_df.drop(category, axis = 1)

amazon_df.head()

In [ ]:
# Check data types and samples of "actual_price" and "discounted_price"
print(f"Dtype Length (Actual Price): {len(amazon_df['actual_price'].map(type).unique())}")
print(f"Dtypes (Actual Price): {amazon_df['actual_price'].map(type).unique()}")
print(f"Sample data (Actual Price): {amazon_df['actual_price'].sample(20).tolist()}")
print(f"Dtype Length (Discounted Price: {len(amazon_df['discounted_price'].map(type).unique())}")
print(f"Dtypes (Discounted Price): {amazon_df['discounted_price'].map(type).unique()}")
print(f"Sample data (Discounted Price): {amazon_df['actual_price'].sample(20).tolist()}")

In [ ]:
# Rename "actual_price" and "discounted_price" to indicate the currency
amazon_df = amazon_df.rename(columns = {"actual_price": "actual_price_inr", "discounted_price": "discounted_price_inr"})

In [ ]:
# Transform the "actual_price_inr" and "discount_price_inr" columns to make them numeric
def transform_price(column):
    amazon_df.loc[:, column] = pd.to_numeric(amazon_df[column].str.replace("₹", "").str.replace(",", ""))

transform_price("actual_price_inr")
transform_price("discounted_price_inr")

amazon_df.head()

In [ ]:
# Define function for currency conversion to USD, the world's most referenced currency
# Update the conversion rate or change the currency as needed
def inr_to_usd(amount_in_inr):
    conversion_rate = 80  # define conversion rate here
    return amount_in_inr / conversion_rate

def replace_column(df, old_col, new_col, new_values):
    # Find the position of the old column
    col_index = df.columns.get_loc(old_col)
    
    # Drop the old column
    df.drop(old_col, axis = 1, inplace = True)
    
    # Insert the new column at the same position
    df.insert(col_index, new_col, new_values)

# Convert prices and replace columns in-place
replace_column(amazon_df, "actual_price_inr", "actual_price", inr_to_usd(amazon_df["actual_price_inr"]))
replace_column(amazon_df, "discounted_price_inr", "discounted_price", inr_to_usd(amazon_df["discounted_price_inr"]))

amazon_df.head()

In [ ]:
# Drop "discount_percentage"
amazon_df = amazon_df.drop(["discount_percentage"], axis = 1)
amazon_df.head()

In [ ]:
# Find unique data types in each column
amazon_df.apply(lambda col: col.map(type).unique())

In [ ]:
# Check rating data
amazon_df["rating"].unique()

In [ ]:
# Check the rows where the "rating" column is equal to "|"
amazon_df[amazon_df["rating"] == "|"]

In [ ]:
# Drop rows where the "rating" column is equal to "|"
amazon_df = amazon_df[amazon_df["rating"] != "|"]

amazon_df

In [ ]:
# Transform the "rating" and "rating_count" column and make them numeric
amazon_df["rating"] = pd.to_numeric(amazon_df["rating"])
amazon_df["rating_count"] = pd.to_numeric(amazon_df["rating_count"].str.replace(",", ""))

amazon_df.head()

In [ ]:
# Check for duplicates
def check_duplicates(dataframe):
    return dataframe.duplicated().sum()

print(check_duplicates(amazon_df))

In [ ]:
# Remove duplicates and reset index
amazon_df = amazon_df[amazon_df.duplicated() == False].reset_index(drop = True)

amazon_df

In [ ]:
# Further check the number of unique "product_id"
amazon_df["product_id"].nunique()

In [ ]:
# Identify duplicate "product_id" to investigate potential issues
amazon_df[amazon_df.duplicated(subset = "product_id", keep=False)].sort_values(by = "product_id", ascending = True)

In [ ]:
# Most rows with duplicate "product_id" are nearly identical, except for minor differences in the "rating_count" and/or "review_content" columns
# To maintain data quality, duplicates should be removed while keeping the row with the highest "rating_count" for each product
amazon_df = amazon_df.sort_values(by = "rating_count", ascending = False) \
                     .drop_duplicates(subset = "product_id", keep = "first") \
                     .sort_index() \
                     .reset_index(drop = True)
amazon_df

In [ ]:
# Analyse Sentiment

# Create the sentiment score column
amazon_df = amazon_df.assign(
    sentiment_score = amazon_df["review_content"].apply(lambda x: TextBlob(str(x)).sentiment.polarity))

# Create the sentiment category column
amazon_df = amazon_df.assign(
    sentiment_category = amazon_df["sentiment_score"].apply(lambda x: "Positive" if x > 0 else ("Negative" if x < 0 else "Neutral")))

amazon_df.head()

In [ ]:
# Drop and rename columns
amazon_df = amazon_df.drop(["category", "about_product", "review_title", "review_content"], axis = 1)
amazon_df = amazon_df.rename(columns = {"category_l01": "category", "category_l02": "subcategory"})

amazon_df.head()

In [ ]:
# Find unique data types in each column
amazon_df.apply(lambda col: col.map(type).unique())

In [ ]:
# Force convert data types
amazon_df = amazon_df.convert_dtypes()

amazon_df.dtypes

In [ ]:
# Export cleaned CSV file
amazon_df.to_csv("data/amazon_etl.csv", index = False)